In [1]:
from bs4 import BeautifulSoup
import sys
import time
import os
import requests
import codecs
import json
import csv
import pandas as pd
import numpy as np
site_url = "https://www1.eeoc.gov/"
base_url = "https://www1.eeoc.gov/eeoc/statistics/employment/jobpat-eeo1"

def get_page(url):
    response = requests.get(url)
    html = response.text.encode('utf-8')
    return html


In [2]:
def get_reports_page(url):
    url = url + "eeoc/statistics/employment/jobpat-eeo1"
    html = get_page(url)
    soup = BeautifulSoup(html, 'html.parser')
    all_reports = soup.find('div', {'class': 'CS_Textblock_Text'}).find('ul').find_all('a')
    for report in all_reports:
        year = report.text.split(" ")
        if year[0] != "Glossary":
#             if int(year[0]) == 2008:
#                 get_industry_page('https://www.eeoc.gov/eeoc/statistics/employment/jobpat-eeo1/2008/nac2/index.html')
#             if int(year[0]) >= 2005 and int(year[0]) <= 2007:
#                 report_url = site_url + report['href']
#                 parse_report_page(report_url, year[0])  
            if int(year[0]) >= 2005 and int(year[0]) < 2007:
                report_url = site_url + report['href']
                parse_report_page(report_url, year[0])    

In [4]:
def parse_recent_report_page(url,year):
    html = get_page(url)
    soup = BeautifulSoup(html, 'html.parser')    
    print(url)

In [5]:
def parse_report_page(url,year):
    html = get_page(url)
    soup = BeautifulSoup(html, 'html.parser')    
    print(year)
    print("here")
    if soup.find('h1', text="This Page Was Not Found.") is None:
        links = soup.find_all('a', text='NAICS-2 Aggregate')
        for link in links:
            industry_url = base_url + "/"+  year  +"/" + link['href']
            print(industry_url)
            get_industry_page(industry_url)
    else:
        print('skip')

    

In [6]:
def get_industry_page(url):
    html = get_page(url)
    soup = BeautifulSoup(html, 'html.parser')

    industries = soup.find('body').find('table').find_all('a',href=True)
    stripped_url = url.split('index.html')
    for industry in industries:
        industry_url = stripped_url[0] + industry['href']
        parse_industry(industry_url)

In [7]:
def parse_industry(url):
#     print(url)
    html = get_page(url)
    splitURl = (os.path.splitext(url)[0].split('/'))
    fileName = splitURl[7] +'-'+ splitURl[8]+'-'+splitURl[9]
    print(fileName)
    parseHTML(html,fileName)

    

In [8]:
def parseHTML(x,fileName):
#   This code parses the table that is available on EEOC website

    soup = BeautifulSoup((x),'html.parser')
    table = (soup.select('table'))[0]
    tableHeader = table.findAll('th',{'scope':'col'})

    headerRow = []
    for x in tableHeader:
        headerRow.append(x.text)
    
    body = (table.select('tbody'))
    temp=[]
    rows = []
    for h in body:
        row = (h.findAll('tr'))
        for r in row:
            temp.append((r.find('th').text))
            values = (r.select('td'))
            for v in values:
                temp.append(v.text)
            rows.append(temp)
            temp=[]

    with open(fileName, 'w') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerow([headerRow[0],headerRow[1],headerRow[2],headerRow[3],headerRow[4],headerRow[5],headerRow[6],headerRow[7],headerRow[8],headerRow[9],headerRow[10]])
        for x in rows:
            if (len(x)==11):
                writer.writerow([x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9],x[10]])

In [9]:
get_reports_page(site_url)

2006
here
https://www1.eeoc.gov/eeoc/statistics/employment/jobpat-eeo1/2006/nac2/index.html
2006-nac2-11
2006-nac2-21
2006-nac2-22
2006-nac2-23
2006-nac2-31
2006-nac2-32
2006-nac2-33
2006-nac2-42
2006-nac2-44
2006-nac2-45
2006-nac2-48
2006-nac2-49
2006-nac2-51
2006-nac2-52
2006-nac2-53
2006-nac2-54
2006-nac2-55
2006-nac2-56
2006-nac2-61
2006-nac2-62
2006-nac2-71
2006-nac2-72
2006-nac2-81
2006-nac2-92
2005
here
https://www1.eeoc.gov/eeoc/statistics/employment/jobpat-eeo1/2005/nac2/index.html
2005-nac2-11
2005-nac2-21
2005-nac2-22
2005-nac2-23
2005-nac2-31
2005-nac2-32
2005-nac2-33
2005-nac2-42
2005-nac2-44
2005-nac2-45
2005-nac2-48
2005-nac2-49
2005-nac2-51
2005-nac2-52
2005-nac2-53
2005-nac2-54
2005-nac2-55
2005-nac2-56
2005-nac2-61
2005-nac2-62
2005-nac2-71
2005-nac2-72
2005-nac2-81
2005-nac2-92


In [13]:
def parseCSVfromWeb():
    rootdir = './'
    folders=[]
    for subdir, dirs, files in os.walk(rootdir):
        for d in dirs:
            if (d[0]=='2'):
                folders.append(d)
#         for f in files:
#             print (f)
#     print (folders)
    
    for f in folders:
        rootdir = './'+ f
#         print (rootdir)
        for subdir, dirs, files in os.walk(rootdir):
            for file in files:
                if (file[:2]=='20'):
                    print (file)
                    f = rootdir + '/' +file
                    getDFfromMyCSV(f)
        #         for d in subdzr:
#             print (d)
#         year = (subdir.split('-')[0][2:])
parseCSVfromWeb()

2005 national agg scraped from web.csv
here
./2005-Data/2005 national agg scraped from web.csv
                            Total Employment Officials & Managers  \
Racial/Ethnic Group and Sex                                         
ALL EMPLOYEES                     44,338,747            4,794,006   
Men                               23,191,888            3,098,966   
Women                             21,146,859            1,695,040   
WHITE                             30,350,518            4,006,438   
Men                               16,070,733            2,632,932   
Women                             14,279,785            1,373,506   
MINORITY                          13,988,229              787,568   
Men                                7,121,155              466,034   
Women                              6,867,074              321,534   
BLACK                              6,232,862              319,520   
Men                                2,821,496              166,877   
Women   

In [12]:
def getDFfromMyCSV(fileName):
#   this code creates a data frame from the CSV that we created 

    print ('here')
    print (fileName)
    df = pd.DataFrame.from_csv(fileName)
    print (df)
#     print (df.drop(df.index[[22,23,24,25,26,27,28]]))
# getDFfromCSV()

In [17]:
def getDFfromGOVCSV(fileName):
#   this code creates a data frame from the CSV file that the government provides

    print (fileName)
    index = ['ALL EMPLOYEES','Men','Women','WHITE','Men','Women','MINORITY',
             'Men','Women','BLACK','Men','Women','HISPANIC','Men','Women',
             'ASIAN AMERICAN','Men', 'Women','AMERICAN INDIAN','Men','Women',
             'HAWAIIAN','Men','Women','TWO OR MORE RACES','Men','Women']
    column = ['Total Employment','Officials & Managers','Professionals','Technicians',
              "Sales Workers",'Office & Clerical Workers','Craft Workers','Operatives',
              'Laborers','Service Workers']
#     myDF = myDF.fillna(0)

    dfFromFile = pd.DataFrame.from_csv(fileName,sep=';')

    
    sAndRace = ['TOTAL','MT','FT','WHT','WHM','WHF','MinT','MinM','MinF','BLKT','BLKM','BLKF','HISPT',
                      'HISPM','HISPF','ASIANT','ASIANM','ASIANF','AIANT','AIANM','AIANF',
                      'nhopiT','NHOPIM','NHOPIF','tomrT','TOMRM','TOMRF']
    table = []
    for x in sAndRace:
        tempRow = []
        for y in range(1,11):
#             print (x,y)
            if (y==1):
                tempRow.append(dfFromFile.get_value(index='UNITED STATES',col=(x +'1')))
            elif (y==2):
                tempRow.append(dfFromFile.get_value(index='UNITED STATES',col=(x+'1_2')))
            elif (y==10):
                tempRow.insert(0,dfFromFile.get_value(index='UNITED STATES',col=(x+'10')))
            else:
                tempRow.append(dfFromFile.get_value(index='UNITED STATES',col=(x+str(y))))
#             print (tempRow)
        table.append(tempRow)
    
#     print (table)
    myDF = pd.DataFrame(data=table, index=index, columns=column)
#     myDF = pd.DataFrame(data=table, index=index)

    print(myDF)

getDFfromGOVCSV('2009-Data/YEAR09_US.txt')
# getDFfromGOVCSV('2007-Data/YEAR07_NAC2.txt')

2009-Data/YEAR09_US.txt
                   Total Employment  Officials & Managers  Professionals  \
ALL EMPLOYEES              47466290                803349        4350710   
Men                        24469000                575917        2695227   
Women                      22997290                227432        1655483   
WHITE                      31280843                711504        3487536   
Men                        16346939                516169        2193795   
Women                      14933904                195335        1293741   
MINORITY                   16185447                 91845         863174   
Men                         8122061                 59748         501432   
Women                       8063386                 32097         361742   
BLACK                       6533334                 24447         307205   
Men                         2892516                 12727         155767   
Women                       3640818                 11720       